In [11]:
#Import libraries and functions

%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import glob #pattern package to find filename
from skimage.io import imread
from scipy import ndimage
from scipy.ndimage import shift
from scipy.ndimage.measurements import center_of_mass
from scipy.ndimage.interpolation import rotate
from scipy.optimize import curve_fit

In [12]:
#Lattice parameters

a = 4.36138
c = 40.92263

lattice_para = np.array([[a, 0, 0], [a*np.cos(2*np.pi/3), a*np.sin(2*np.pi/3), 0], [0, 0, c]])

ra = 4*np.pi/(np.sqrt(3)*a)
rc = 2*np.pi/c

rec_lattice_para = np.array([[ra*np.cos(np.pi/6), ra*np.sin(np.pi/6), 0], [ra*np.cos(5*np.pi/6), ra*np.sin(5*np.pi/6), 0], [0, 0, rc]])

In [8]:
#Defining some functions

##Functions that return Q, K, and q for a given h, k, l

def Q(h, k, l):
    return h*rec_lattice_para[0] + k*rec_lattice_para[1] + l*rec_lattice_para[2]

def K(h, k, l):
    d_test = 10 #this will store the minimum norm for comparison (10 is just an arbitrary number)
    m_test = 1 #this will store the integer m corresponding to minimum norm
    n_test = 1 #this will store the integer n corresponding to minimum norm
    for m in np.arange(np.rint(h)-2, np.rint(h)+2, 1):
        for n in np.arange(np.rint(k)-2, np.rint(k)+2, 1):
            d = (h - m)**2 + (k - n)**2 - (h - m)*(k - n)
            if d < d_test:
                d_test = d
                m_test = m
                n_test = n
    #print(d_test)
    #print(m_test)
    #print(n_test)
    return m_test*rec_lattice_para[0] + n_test*rec_lattice_para[1] + np.rint(l)*rec_lattice_para[2]
            
def q(h, k, l):
    return Q(h, k, l) - K(h, k, l) 

##Function that returns r = xa + yb + zc for a given x, y, z

def R(x, y, z):
    return x*lattice_para[0] + y*lattice_para[1] + z*lattice_para[2]

##Exception for division by zero

def div(n, d):
    return n/d if d > 0.15 else 0

def w(h, k, l):
    return (np.linalg.norm(q(h, k, l)))**1

def e_t(h, k, l):
    e_t_temp = np.cross(q(h, k, l), [0, 0, 1])
    if np.linalg.norm(e_t_temp) != 0:
        return e_t_temp/np.linalg.norm(e_t_temp)
    else:
        return e_t_temp
    
def e_l(h, k, l):
    e_l_temp = q(h, k, l)
    if np.linalg.norm(e_l_temp) != 0:
        return e_l_temp/np.linalg.norm(e_l_temp)
    else:
        return e_l_temp    

##Function that outputs the structure factor

def F(h, k, l, B, r):
    f_sum = 0
    for i in np.arange(0, np.shape(r)[0]): #sum over all the atoms in unit cell
        f_sum = f_sum + np.exp(-B[i]*(np.linalg.norm(Q(h, k, l))**2))*np.dot(Q(h, k, l), e_l(h, k, l))*np.exp(-1j*np.dot(K(h, k, l), r[i]))
    return f_sum    

In [9]:
#Importing unit cell coordinates, phonon eigenvectors, and other material parameters

B = [0.05, 0.05, 0.05]

# e = 0.1*np.array([[R(1, 0, 0), R(1, 0, 0), R(1, 0, 0)],
#               [R(0, 1, 0), R(0, 1, 0), R(0, 1, 0)],
#               [R(-1, -1, 0), R(-1, -1, 0), R(-1, -1, 0)]])

unit_r = np.array([R(0, 0, 0), R(2/3, 1/3, 1/3), R(1/3, 2/3, 2/3)])


# e = np.array([[R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0), R(1, 0, 0)],
#              [R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0), R(0, 1, 0)],
#              [R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0), R(-1, 1, 0)]])

# unit_r = np.array([R(0, 0, 0), R(0.67, 0.33, 0.33), R(0.33, 0.67, 0.67), R(0, 0, 0.42), R(0, 0, 0.57), R(0.67, 0.33, 0.75), R(0.67, 0.33, 0.91), R(0.33, 0.67, 0.09), R(0.33, 0.67, 0.24), R(0, 0, 0.13), R(0, 0, 0.86), R(0.67, 0.33, 0.47), R(0.67, 0.33, 0.2), R(0.33, 0.67, 0.8), R(0.33, 0.67, 0.53), R(0, 0, 0.3), R(0, 0, 0.7), R(0.67, 0.33, 0.63), R(0.67, 0.33, 0.04), R(0.33, 0.67, 0.96), R(0.33, 0.67, 0.37)])

In [10]:
##Create arrays with x, y for integer h, k, l

data_int = np.zeros(((h_f-h_i)*(k_f-k_i),3))

i = 0

for h in np.arange(h_i, h_f, 1):
    for k in np.arange(k_i, k_f, 1):
        data_int[i] = [Q(h, k, 0)[0], Q(h, k, 0)[1], 0]
        #print(data_int[i])
        i = i + 1
        
x_int = data_int[:,0]
y_int = data_int[:,1]

##Create arrays with x, y for integer h, k, l satisfying Bragg condition

data_int_bragg = np.zeros(((h_f-h_i)*(k_f-k_i),3))

i = 0

for h in np.arange(h_i, h_f, 1):
    for k in np.arange(k_i, k_f, 1):
        
        ###Sum over phase factors to check for selection rule
        temp_sum = 0
        for j in np.arange(0, np.shape(unit_r)[0]): #sum over all the atoms in unit cell
            temp_sum = temp_sum + np.exp(-1j*np.dot(K(h, k, 0), unit_r[j]))
        
        if np.abs(temp_sum) > 0.001:
            data_int_bragg[i] = [Q(h, k, 0)[0], Q(h, k, 0)[1], 0]
            #print(data_int[i])
        i = i + 1

x_int_bragg = data_int_bragg[:,0]
y_int_bragg = data_int_bragg[:,1]

##Create mesh of diffuse scattering intensity data for entire scattering space

h_i = -5
h_f = 5
k_i = -5
k_f = 5
mesh = 250

data = np.zeros((mesh*mesh,3))

i = 0

for h in np.arange(h_i, h_f, (h_f-h_i)/mesh):
    for k in np.arange(k_i, k_f, (k_f-k_i)/mesh):
        data[i] = [Q(h, k, 0)[0], Q(h, k, 0)[1], 0] 
        data[i,2] = div(np.abs(F(h, k, 0, B, unit_r)), w(h, k, 0))
        i = i + 1

x = data[:,0]
y = data[:,1]
z = data[:,2]

##Plot total intensity

fig1, ax1 = plt.subplots()

tcf = ax1.tricontourf(x, y, z, levels = 20)
fig1.colorbar(tcf)

ax1.scatter(x_int_bragg, y_int_bragg, s = 1, color = 'w')

plt.xlim(-5,5)
plt.ylim(-5,5)

ax1.set_aspect('equal')
ax1.set_title('Simulated diffuse scattering')

NameError: name 'h_f' is not defined

In [462]:
print(w(5.2, 2.4, 0))
print(q(5.2, 2.4, 0))
print(q(0.2, 0.4, 0))

9.544544831942309
[ 4.03379638 -8.65024987  0.        ]
[ -0.28812831 -14.47253344   0.        ]


In [568]:
np.dot(np.cross(q(1, 2.4, 3), [0, 0, 1]), Q(1.2, 3, 0))

1.1503301082997064

In [361]:
a = np.array([[0, 1, 1], [1, 0, 0]])
b = np.array([1, 2, 3])
print(a.dot(b))

[5 1]


In [708]:
def F_test(h, k, l, B, r):
    f_sum = 0
    for i in np.arange(0, np.shape(r)[0]): #sum over all the atoms in unit cell
        f_sum = f_sum + np.exp(-1j*np.dot(K(h, k, l), r[i]))
        print(f_sum)
    return f_sum    

np.abs(F_test(3, 0, 0, B, unit_r))

(1+0j)
(2-1.2864981197413093e-15j)
(3-1.929747179611964e-15j)


3.0

In [701]:
np.exp(-1j*np.dot(K(1.1, 1.1, 0), unit_r))

array([-0.88420546+0.46709817j, -0.5       -0.8660254j ,
       -0.76446804+0.64466163j])